First, let's generate some sample data to work with.     
We use `np.zeros` to generate random trajectory data divided into multiple roads with nodes. The `estimate_success_probability` function now takes a 3D array of trajectory data as input, where the **first dimension corresponds to the road**, the **second dimension corresponds to the nod**e, and the **third dimension corresponds to the x and y coordinates of the node**.

In [5]:
# !pip install pgmpy

In [8]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator

# simulate observed data for 100 nodes
nodes = np.arange(100)
success_rates = np.random.rand(100)

# construct a Bayesian network with the observed nodes as parents
model = BayesianModel()
model.add_nodes_from(nodes)
for i in range(1, len(nodes)):
    model.add_edge(i-1, i)

# estimate the parameters of the model using Maximum Likelihood Estimation (MLE)
estimator = MaximumLikelihoodEstimator(model, data=pd.DataFrame({'success_rate': success_rates}))
cpds = estimator.get_parameters()

# use Bayesian inference to estimate the success rates of unobserved nodes
for i in range(100, 1000):
    cpd = BayesianEstimator.estimate_cpd(model, i, estimator)
    cpds.append(cpd)
    evidence = {j: {'success_rate': success_rates[j]} for j in range(i-1, i-101, -1)}
    query = model.predict_probability({'success_rate': None}, evidence=evidence)
    success_rates[i] = query['success_rate'].values[0]

# print the estimated success rates of the nodes
print(success_rates)


ImportError: DLL load failed while importing _ufuncs: The specified module could not be found.

In [3]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a graph
G = nx.Graph()

# Add nodes to the graph
for i in range(1000):
    G.add_node(i)

# Add edges (roads) to the graph
for i in range(999):
    success_prob = 0.5  # replace with your simulated success rate for this road
    G.add_edge(i, i+1, weight=success_prob)

# Get the positions of the nodes for plotting
pos = nx.spring_layout(G)

# Get the success probabilities and edge weights for plotting
edge_weights = [G[u][v]['weight'] for u, v in G.edges()]
success_probs = [(1 - w, w, 0.0) for w in edge_weights]

# Plot the graph
nx.draw(G, pos=pos, edge_color=success_probs, width=2, edge_cmap=plt.cm.RdYlGn_r)
plt.show()


AttributeError: module 'scipy.sparse' has no attribute 'coo_array'

Next, let's assume we have some weather data, such as temperature and precipitation, for each day in our dataset:

In [2]:
import pandas as pd 
# Generate random weather data
dates = pd.date_range(start='2020-01-01', end='2022-12-31', freq='D')
weather_data = pd.DataFrame({
    'date': dates,
    'temperature': np.random.normal(loc=20, scale=5, size=len(dates)),
    'precipitation': np.random.normal(loc=0, scale=1, size=len(dates)),
})

We can now construct a Bayesian network to model the relationships between the various factors that affect the probability of successful arrival. 
* We'll use the PyMC3 library to define and sample from the posterior distribution of the network parameters:

In [4]:
# !pip install pymc3

* This model assumes that the probability of successful arrival for each road segment depends on the temperature and precipitation on that day.
* We've defined priors for the conditional probabilities of success, temperature, and precipitation, and used these to calculate the overall probability of success for each road segment. 
* We then define the likelihood function using the observed success flags, and use PyMC3 to sample from the posterior distribution of the network parameters.

In [5]:
import pymc3 as pm

# Define the model
with pm.Model() as model:
    # Define priors for the conditional probabilities
    p_success = pm.Beta('p_success', alpha=2, beta=2, shape=(100,))
    p_temp = pm.Normal('p_temp', mu=20, sigma=5, shape=(100,))
    p_precip = pm.Normal('p_precip', mu=0, sigma=1, shape=(100,))
    
    # Define the conditional probability of success given the road segment, temperature, and precipitation
    p = pm.Deterministic('p', p_success[start_locations] * p_temp[start_locations] * p_precip[start_locations])
    
    # Define the likelihood function
    y = pm.Bernoulli('y', p=p, observed=success_flags)
    
    # Sample from the posterior distribution
    trace = pm.sample(1000, tune=1000)


WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\anaconda\\lib\\site-packages\\xarray-0.20.1.dist-info\\METADATA'